Загружаем датасет

In [20]:
import pandas as pd

pokemon = pd.read_csv("../data/Pokemon_preprocessed.csv", index_col=0)
pokemon.info()
pokemon.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800 entries, 0 to 799
Data columns (total 28 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        800 non-null    object
 1   Total       800 non-null    int64 
 2   HP          800 non-null    int64 
 3   Attack      800 non-null    int64 
 4   Defense     800 non-null    int64 
 5   Sp. Atk     800 non-null    int64 
 6   Sp. Def     800 non-null    int64 
 7   Speed       800 non-null    int64 
 8   Generation  800 non-null    int64 
 9   Legendary   800 non-null    bool  
 10  t_Bug       800 non-null    int64 
 11  t_Dark      800 non-null    int64 
 12  t_Dragon    800 non-null    int64 
 13  t_Electric  800 non-null    int64 
 14  t_Fairy     800 non-null    int64 
 15  t_Fighting  800 non-null    int64 
 16  t_Fire      800 non-null    int64 
 17  t_Flying    800 non-null    int64 
 18  t_Ghost     800 non-null    int64 
 19  t_Grass     800 non-null    int64 
 20  t_Ground  

,Name,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,...,t_Ghost,t_Grass,t_Ground,t_Ice,t_Normal,t_Poison,t_Psychic,t_Rock,t_Steel,t_Water
0,Bulbasaur,318,45,49,49,65,65,45,1,False,...,0,1,0,0,0,1,0,0,0,0
1,Ivysaur,405,60,62,63,80,80,60,1,False,...,0,1,0,0,0,1,0,0,0,0
2,Venusaur,525,80,82,83,100,100,80,1,False,...,0,1,0,0,0,1,0,0,0,0
3,VenusaurMega Venusaur,625,80,100,123,122,120,80,1,False,...,0,1,0,0,0,1,0,0,0,0
4,Charmander,309,39,52,43,60,50,65,1,False,...,0,0,0,0,0,0,0,0,0,0


Разбиваем данные на параметры входные и целевые.

In [26]:
X = pokemon.drop(["Legendary","Name"], axis=1).values
Y = pokemon.iloc[:, 9].values

In [27]:
print(X)

[[318  45  49 ...   0   0   0]
 [405  60  62 ...   0   0   0]
 [525  80  82 ...   0   0   0]
 ...
 [600  80 110 ...   0   0   0]
 [680  80 160 ...   0   0   0]
 [600  80 110 ...   0   0   1]]


In [11]:
print(Y)

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
  True  True  True False False False  True  True  T

Разбиваем данные на обучающие и тестовые.

In [97]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)

In [98]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Метод k-ближайших соседей (K-Nearest Neighbors)

## Классификация

In [94]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=6)
clf.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=6)

In [95]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[142   5]
 [  7   6]]
              precision    recall  f1-score   support

       False       0.95      0.97      0.96       147
        True       0.55      0.46      0.50        13

    accuracy                           0.93       160
   macro avg       0.75      0.71      0.73       160
weighted avg       0.92      0.93      0.92       160



## Поиск гиперпараметра

Найдем наиболее подходящие параметры

In [99]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

parameters = {'n_neighbors':[1, 2, 3, 4, 5, 6, 7, 8]}
knn = KNeighborsClassifier()
clf = GridSearchCV(knn, parameters)
clf.fit(X_train, y_train)
clf.best_params_

{'n_neighbors': 3}

Наилучший параметр соседей - 3.

## Кросс-валидация

Проверяем устойчивость данных и используем полученный гиперпараметр.

In [101]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X, Y):
  print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = Y[train_index], Y[test_index]

TRAIN: [147 148 149 150 151 152 153 154 155 159 160 161 165 166 167 168 169 170
 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188
 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206
 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242
 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260
 261 265 266 267 268 271 272 273 274 275 276 277 278 279 280 281 282 283
 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301
 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319
 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337
 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355
 356 357 358 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373
 374 375 376 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391
 392 393 394 395 396 397 398 399 400 401 402

In [102]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X, Y):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = Y[train_index], Y[test_index]
  clf = KNeighborsClassifier(n_neighbors=3)
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(classification_report(y_test, y_pred))
  print(clf.score(X_test, y_test))

              precision    recall  f1-score   support

       False       0.96      0.97      0.97       147
        True       0.64      0.54      0.58        13

    accuracy                           0.94       160
   macro avg       0.80      0.76      0.77       160
weighted avg       0.93      0.94      0.94       160

0.9375
              precision    recall  f1-score   support

       False       0.99      0.97      0.98       147
        True       0.71      0.92      0.80        13

    accuracy                           0.96       160
   macro avg       0.85      0.94      0.89       160
weighted avg       0.97      0.96      0.96       160

0.9625
              precision    recall  f1-score   support

       False       0.98      0.95      0.97       147
        True       0.59      0.77      0.67        13

    accuracy                           0.94       160
   macro avg       0.78      0.86      0.82       160
weighted avg       0.95      0.94      0.94       160

0.937

# Классификатор дерева решений (Decision Tree Classifier)

## Классификация

In [68]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

In [69]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[144   4]
 [  2  10]]
              precision    recall  f1-score   support

       False       0.99      0.97      0.98       148
        True       0.71      0.83      0.77        12

    accuracy                           0.96       160
   macro avg       0.85      0.90      0.87       160
weighted avg       0.97      0.96      0.96       160



## Поиск гиперпараметра

In [79]:
from sklearn.model_selection import GridSearchCV

parameters = { "max_depth": range(1,20),
              'max_leaf_nodes': range(2, 10),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2) }

trc = tree.DecisionTreeClassifier()
clf = GridSearchCV(trc, parameters, cv=5)
clf.fit(X_train, y_train)
clf.best_params_

{'max_depth': 5,
 'max_leaf_nodes': 7,
 'min_samples_leaf': 6,
 'min_samples_split': 8}

## Кросс-Валидация

In [86]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X, Y):
  print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = Y[train_index], Y[test_index]

TRAIN: [147 148 149 150 151 152 153 154 155 159 160 161 165 166 167 168 169 170
 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188
 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206
 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242
 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260
 261 265 266 267 268 271 272 273 274 275 276 277 278 279 280 281 282 283
 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301
 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319
 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337
 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355
 356 357 358 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373
 374 375 376 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391
 392 393 394 395 396 397 398 399 400 401 402

In [103]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X, Y):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = Y[train_index], Y[test_index]
  clf = tree.DecisionTreeClassifier(max_depth=5, max_leaf_nodes=7, min_samples_leaf=6, min_samples_split=8)
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(classification_report(y_test, y_pred))
  print(clf.score(X_test, y_test))

              precision    recall  f1-score   support

       False       0.93      1.00      0.96       147
        True       1.00      0.15      0.27        13

    accuracy                           0.93       160
   macro avg       0.97      0.58      0.62       160
weighted avg       0.94      0.93      0.91       160

0.93125
              precision    recall  f1-score   support

       False       0.96      0.99      0.98       147
        True       0.88      0.54      0.67        13

    accuracy                           0.96       160
   macro avg       0.92      0.77      0.82       160
weighted avg       0.95      0.96      0.95       160

0.95625
              precision    recall  f1-score   support

       False       0.99      0.96      0.98       147
        True       0.67      0.92      0.77        13

    accuracy                           0.96       160
   macro avg       0.83      0.94      0.87       160
weighted avg       0.97      0.96      0.96       160

0.9

## Наивный байесовский классификатор (Naive Bayes)

## Классификация

In [42]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB()

In [43]:
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[76 72]
 [ 0 12]]
              precision    recall  f1-score   support

       False       1.00      0.51      0.68       148
        True       0.14      1.00      0.25        12

    accuracy                           0.55       160
   macro avg       0.57      0.76      0.46       160
weighted avg       0.94      0.55      0.65       160



## Поиск гиперпараметра

In [108]:
from sklearn.model_selection import GridSearchCV
import numpy as np

parameters = { 'var_smoothing': np.logspace(0,-9, num=100)}

gnb = GaussianNB()
clf = GridSearchCV(gnb, parameters, cv=5)
clf.fit(X_train, y_train)
clf.best_params_

{'var_smoothing': 0.43287612810830584}

## Кросс-Валидация

In [109]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X, Y):
  print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = Y[train_index], Y[test_index]

TRAIN: [147 148 149 150 151 152 153 154 155 159 160 161 165 166 167 168 169 170
 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188
 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206
 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242
 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260
 261 265 266 267 268 271 272 273 274 275 276 277 278 279 280 281 282 283
 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301
 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319
 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337
 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355
 356 357 358 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373
 374 375 376 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391
 392 393 394 395 396 397 398 399 400 401 402

In [111]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X, Y):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = Y[train_index], Y[test_index]
  clf = GaussianNB(var_smoothing = 0.43287612810830584)
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(classification_report(y_test, y_pred))
  print(clf.score(X_test, y_test))

              precision    recall  f1-score   support

       False       0.95      1.00      0.97       147
        True       1.00      0.38      0.56        13

    accuracy                           0.95       160
   macro avg       0.97      0.69      0.76       160
weighted avg       0.95      0.95      0.94       160

0.95
              precision    recall  f1-score   support

       False       0.97      0.99      0.98       147
        True       0.89      0.62      0.73        13

    accuracy                           0.96       160
   macro avg       0.93      0.80      0.85       160
weighted avg       0.96      0.96      0.96       160

0.9625
              precision    recall  f1-score   support

       False       0.95      0.98      0.96       147
        True       0.62      0.38      0.48        13

    accuracy                           0.93       160
   macro avg       0.79      0.68      0.72       160
weighted avg       0.92      0.93      0.92       160

0.93125

## Метод опорных векторов (Support Vector Machines)

## Классификация

In [44]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)

SVC()

In [45]:
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[145   3]
 [  5   7]]
              precision    recall  f1-score   support

       False       0.97      0.98      0.97       148
        True       0.70      0.58      0.64        12

    accuracy                           0.95       160
   macro avg       0.83      0.78      0.80       160
weighted avg       0.95      0.95      0.95       160



## Поиск гиперпараметра

In [113]:
from sklearn.model_selection import GridSearchCV
import numpy as np

parameters = { 'kernel':('linear', 'rbf'),'C': [1, 10], 'gamma': [0.001, 0.01, 1]}

svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(X_train, y_train)
clf.best_params_

{'C': 1, 'gamma': 0.001, 'kernel': 'linear'}

## Кросс-Валидация

In [114]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X, Y):
  print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = Y[train_index], Y[test_index]

TRAIN: [147 148 149 150 151 152 153 154 155 159 160 161 165 166 167 168 169 170
 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188
 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206
 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242
 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260
 261 265 266 267 268 271 272 273 274 275 276 277 278 279 280 281 282 283
 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301
 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319
 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337
 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355
 356 357 358 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373
 374 375 376 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391
 392 393 394 395 396 397 398 399 400 401 402

In [115]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X, Y):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = Y[train_index], Y[test_index]
  clf = svm.SVC(C = 1, gamma = 0.001, kernel = 'linear')
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(classification_report(y_test, y_pred))
  print(clf.score(X_test, y_test))

              precision    recall  f1-score   support

       False       0.95      0.99      0.97       147
        True       0.83      0.38      0.53        13

    accuracy                           0.94       160
   macro avg       0.89      0.69      0.75       160
weighted avg       0.94      0.94      0.93       160

0.94375
              precision    recall  f1-score   support

       False       0.97      0.99      0.98       147
        True       0.80      0.62      0.70        13

    accuracy                           0.96       160
   macro avg       0.88      0.80      0.84       160
weighted avg       0.95      0.96      0.95       160

0.95625
              precision    recall  f1-score   support

       False       0.96      0.97      0.97       147
        True       0.64      0.54      0.58        13

    accuracy                           0.94       160
   macro avg       0.80      0.76      0.77       160
weighted avg       0.93      0.94      0.94       160

0.9

## Логистическая регрессия (Logistic Regression)

## Классификация

In [47]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

In [48]:
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[142   6]
 [  5   7]]
              precision    recall  f1-score   support

       False       0.97      0.96      0.96       148
        True       0.54      0.58      0.56        12

    accuracy                           0.93       160
   macro avg       0.75      0.77      0.76       160
weighted avg       0.93      0.93      0.93       160



## Подбор гиперпараметров

In [116]:
from sklearn.model_selection import GridSearchCV

parameters = {'penalty':['l1','l2'],'dual':[True,False],'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000], 'fit_intercept':[True,False],
        'solver':['saga']}

lgr = LogisticRegression()

clf = GridSearchCV(lgr, parameters)

clf.fit(X_train, y_train)

clf.best_params_

C:\Users\Power\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Power\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Power\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\Power\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 445, in _check_solver
    raise ValueError("Solver %s supports only "
ValueError: Solver saga supports only dual=False, got dual=True

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\Users\Power\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter wa

C:\Users\Power\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Power\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Power\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Power\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Power\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which mean

C:\Users\Power\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Power\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Power\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Power\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Power\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver,

C:\Users\Power\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Power\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Power\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Power\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Power\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which mean

{'C': 0.001,
 'dual': False,
 'fit_intercept': True,
 'penalty': 'l1',
 'solver': 'saga'}

## Кросс-Валидация

In [114]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X, Y):
  print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = Y[train_index], Y[test_index]

TRAIN: [147 148 149 150 151 152 153 154 155 159 160 161 165 166 167 168 169 170
 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188
 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206
 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242
 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260
 261 265 266 267 268 271 272 273 274 275 276 277 278 279 280 281 282 283
 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301
 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319
 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337
 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355
 356 357 358 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373
 374 375 376 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391
 392 393 394 395 396 397 398 399 400 401 402

In [119]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X, Y):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = Y[train_index], Y[test_index]
  clf = LogisticRegression(C = 0.001, dual = False, fit_intercept = True,penalty = 'l1', solver = 'saga')
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(classification_report(y_test, y_pred))
  print(clf.score(X_test, y_test))

              precision    recall  f1-score   support

       False       0.92      1.00      0.96       147
        True       0.00      0.00      0.00        13

    accuracy                           0.92       160
   macro avg       0.46      0.50      0.48       160
weighted avg       0.84      0.92      0.88       160

0.91875
              precision    recall  f1-score   support

       False       0.92      1.00      0.96       147
        True       0.00      0.00      0.00        13

    accuracy                           0.92       160
   macro avg       0.46      0.50      0.48       160
weighted avg       0.84      0.92      0.88       160

0.91875
              precision    recall  f1-score   support

       False       0.92      1.00      0.96       147
        True       0.00      0.00      0.00        13

    accuracy                           0.92       160
   macro avg       0.46      0.50      0.48       160
weighted avg       0.84      0.92      0.88       160

0.9

C:\Users\Power\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Power\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Power\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Power\anaconda3\lib\site-packages\sklearn\metr